In [1]:
# 加载必要的包
from os import makedirs, path
import os
import numpy as np
import pandas as pd
from IPython.display import display
from nibabel import save
from nilearn import image, plotting, reporting
from nimare import correct, io, meta, utils #主要用nimare完成元分析
from scipy.stats import norm

# We are now ready to perform the actual ALE analyses with NiMARE. 
# We write a custom function which takes a single Sleuth text file as its input and (a) calculates the ALE map, 
# (b) corrects for multiple comparisons using a Monte Carlo-based FWE correction, 
# and (c) stores the cluster level-thresholded maps into the output directory. 
# We then apply this function to all the Sleuth files we have created in the previous step. 


In [18]:
import nibabel as nib

img = None
try:
    img = nib.load('/Users/ss/Downloads/first_level_stats_output_10_subj/first_level_model_corrected_gii/sub-011/stats_results/run-02/sub-011_task-MGT_run-02_strongly_accept\ -\ strongly_reject_zmap.L.func.gii ')
    print(f"{img} 文件可以正常读取")
except Exception as e:
    print(f"{img} 文件可能已损坏: {e}")


None 文件可能已损坏: No such file or no access: '/Users/ss/Downloads/first_level_stats_output_10_subj/first_level_model_corrected_gii/sub-011/stats_results/run-02/sub-011_task-MGT_run-02_strongly_accept\ -\ strongly_reject_zmap.L.func.gii '


<>:5: SyntaxWarning: invalid escape sequence '\ '
<>:5: SyntaxWarning: invalid escape sequence '\ '
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_4171/3920738479.py:5: SyntaxWarning: invalid escape sequence '\ '
  img = nib.load('/Users/ss/Downloads/first_level_stats_output_10_subj/first_level_model_corrected_gii/sub-011/stats_results/run-02/sub-011_task-MGT_run-02_strongly_accept\ -\ strongly_reject_zmap.L.func.gii ')


In [12]:
# 确保输出目录存在
output_dir = "/Users/ss/Documents/Psych_ALE_meta/results/ale/"
os.makedirs(output_dir, exist_ok=True)

In [13]:
# 定义运行 ALE 分析的函数
def run_ale(text_file, voxel_thresh, cluster_thresh, random_seed, n_iters, output_dir):
    # 设置随机种子
    if random_seed:
        np.random.seed(random_seed)
    
    # 加载 Sleuth 文件并进行 ALE 分析
    dset = io.convert_sleuth_to_dataset(text_file=text_file, target="ale_2mm")
    ale = meta.cbma.ALE()
    res = ale.fit(dset)

    # 使用蒙特卡洛方法进行 FWE 校正
    corr = correct.FWECorrector(
        method="montecarlo", voxel_thresh=voxel_thresh, n_iters=n_iters
    )
    cres = corr.transform(result=res)

    # 保存未阈值化的 z 和 ALE 映射
    prefix = os.path.basename(text_file).replace(".txt", "")
    res.save_maps(output_dir=output_dir, prefix=prefix)
    cres.save_maps(output_dir=output_dir, prefix=prefix)

    # 生成集群级阈值化的 z 和 ALE 映射
    img_clust = cres.get_map("z_level-cluster_corr-FWE_method-montecarlo")
    img_z = cres.get_map("z")
    img_ale = cres.get_map("stat")
    cluster_thresh_z = norm.ppf(1 - cluster_thresh / 2)
    img_clust_thresh = image.threshold_img(img=img_clust, threshold=cluster_thresh_z)
    img_mask = image.math_img("np.where(img > 0, 1, 0)", img=img_clust_thresh)
    img_z_thresh = image.math_img("img1 * img2", img1=img_mask, img2=img_z)
    img_ale_thresh = image.math_img("img1 * img2", img1=img_mask, img2=img_ale)

    # 保存阈值化的映射
    save(img=img_z_thresh, filename=os.path.join(output_dir, prefix + "_z_thresh.nii.gz"))
    save(img=img_ale_thresh, filename=os.path.join(output_dir, prefix + "_stat_thresh.nii.gz"))

    print(f"Analysis complete for '{text_file}'.")

In [16]:
# Ensure cres is defined by running the ALE analysis first
run_ale(
    text_file="/Users/ss/Documents/Psych_ALE_meta/data/Self_all.txt",
    voxel_thresh=0.001,
    cluster_thresh=0.01,
    random_seed=1234,
    n_iters=100,
    output_dir=output_dir
)

print(cres.maps.keys())


INFO:nimare.correct:Using correction method implemented in Estimator: nimare.meta.cbma.ale.ALE.correct_fwe_montecarlo.


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:nimare.meta.cbma.base:Using null distribution for voxel-level FWE correction.


ValueError: No map with name 'z_level-cluster_corr-FWE_method-montecarlo' found.

In [14]:
# 执行 ALE 分析
run_ale(
    text_file="/Users/ss/Documents/Psych_ALE_meta/data/Self_all.txt",
    voxel_thresh=0.001,
    cluster_thresh=0.01,
    random_seed=1234,
    n_iters=100,
    output_dir=output_dir
)

INFO:nimare.correct:Using correction method implemented in Estimator: nimare.meta.cbma.ale.ALE.correct_fwe_montecarlo.


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:nimare.meta.cbma.base:Using null distribution for voxel-level FWE correction.


ValueError: No map with name 'z_level-cluster_corr-FWE_method-montecarlo' found.

In [11]:
def run_ale(text_file, voxel_thresh, cluster_thresh, random_seed, n_iters, output_dir):

    # Let's show the user what we are doing
    # 显示当前操作的信息
    print("ALE ANALYSIS FOR '" + text_file + "' WITH " + str(n_iters) + " PERMUTATIONS")

    # Set a random seed to make the results reproducible
    # 设置随机种子以使结果可重现
    if random_seed:
        np.random.seed(random_seed)

    # Perform the ALE
    # 执行ALE分析
    # 使用io.convert_sleuth_to_dataset函数将Sleuth格式的数据转换为ALE分析所需的数据集。
    dset = io.convert_sleuth_to_dataset(text_file=text_file, target="ale_2mm") # 这里的target是指定的空间模板，可以是MNI152_2mm
    # 创建ALE分析对象`ale`
    ale = meta.cbma.ALE()
    # 使用`ale.fit`方法对数据集进行拟合
    res = ale.fit(dset)

    # FWE correction for multiple comparisons
    # 多重比较的FWE校正
    corr = correct.FWECorrector(
        method="montecarlo", voxel_thresh=voxel_thresh, n_iters=n_iters
    )
    cres = corr.transform(result=res)

    # Save unthresholded maps to the ouput directory
    # 将未阈值化的地图保存到输出目录
    prefix = path.basename(text_file).replace(".txt", "")
    res.save_maps(output_dir=output_dir, prefix=prefix)
    cres.save_maps(output_dir=output_dir, prefix=prefix)

    # Create cluster-level thresholded z and ALE maps
    # 创建集群级阈值化的z和ALE地图
    img_clust = cres.get_map("z_level-cluster_corr-FWE_method-montecarlo")
    img_z = cres.get_map("z")
    img_ale = cres.get_map("stat")

    # 计算cluster阈值的z值 
    cluster_thresh_z = norm.ppf(1 - cluster_thresh / 2) 
    # 使用`image.threshold_img`函数对cluster图像进行阈值化
    img_clust_thresh = image.threshold_img(img=img_clust, threshold=cluster_thresh_z)

    # Create thresholded z and ALE maps
    # 创建一个掩码图像`img_mask`，其中值大于0的体素被设置为1，其余为0。
    img_mask = image.math_img("np.where(img > 0, 1, 0)", img=img_clust_thresh)
    img_z_thresh = image.math_img("img1 * img2", img1=img_mask, img2=img_z)
    # 使用`image.math_img`方法将掩码应用于z图像和ALE图像，得到阈值化的z图像和ALE图像。
    img_ale_thresh = image.math_img("img1 * img2", img1=img_mask, img2=img_ale)

    # Save thresholded maps to the output directory
    # 保存阈值化的图像
    save(img=img_z_thresh, filename=output_dir + "/" + prefix + "_z_thresh.nii.gz")
    save(img=img_ale_thresh, filename=output_dir + "/" + prefix + "_stat_thresh.nii.gz")

In [ ]:

# Finally, let's look at some exemplary results by plotting the (cluster-level FWE-corrected) *z* score map from the main analysis (including all semantic experiments). We also print a table of the corresponding cluster statistics.

if __name__ == "__main__":

    # Glass brain example
    img = image.load_img("data/health_z.nii.gz")
    p = plotting.plot_glass_brain(img, display_mode="lyrz", colorbar=True)

    # Cluster table example
    t = reporting.get_clusters_table(img, stat_threshold=0, min_distance=1000)
    display(t)

    print(cres.maps.keys())
